In [1]:
#Import necessary packages
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder,DeployTransactionBuilder,MessageTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
import requests
import json
import pickle
import csv
import os

from iconsdk.libs.in_memory_zip import gen_deploy_data_content

In [3]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
SCORE_ADDRESS = "cx38d0c023d42c7905ac7b04a8970a0f3ef28800e8"
NEWTON_ADDRESS = "hx43e1f6be06a62a293d06867adc27e7acc9affe69"
DEPLOY_PARAMS = {}

In [4]:
#Start ICON service and wallet
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
#icon_service = IconService(HTTPProvider("http://localhost:9000", 3))

# Returns block information by block height
block = icon_service.get_block(1000)
block


{'version': '0.3',
 'height': 1000,
 'signature': 'Oj2VRhXDnd/3IdrE0ClPKEgGLkYLZt/p/o7D9rlspLoSAEA1vYSZyaOoF5n1QV8hpZaObMuZBGaRm13mjYmcHgA=',
 'prev_block_hash': 'cdd0017615d2af92227ec54d3147b199ba1417e1e6ec99cf33e7005a2b709692',
 'merkle_tree_root_hash': '0000000000000000000000000000000000000000000000000000000000000000',
 'time_stamp': 1586159775591512,
 'confirmed_transaction_list': [],
 'block_hash': '95ea963d180936776a650b6b35640c20c1ca951e26a0b55cb11b1b555ecb559b',
 'peer_id': 'hxaad52424d4aec9dac7d9f6796da527f471269d2c',
 'next_leader': 'hxec79e9c1c882632688f8c8f9a07832bcabe8be8f'}

In [5]:
operator = KeyWallet.load('config/yeouido/keystores/operator', 'bgsa199@#')
prajwol = KeyWallet.load('config/yeouido/keystores/testPrajwol', 'bgsa199@#')

print(operator.get_address())
print(icon_service.get_balance(operator.get_address()))
print(icon_service.get_balance(prajwol.get_address()))

hx8b94a3792f336b71937709dae5487166c180c87a
14003397363390000000000
0


In [55]:
DEPLOY_PARAMS =  {
            "_tokenName": "TestToken",
            "_symbolName": "TK",
			"_initialSupply": 1000,
            "_decimals": 18
		}
        
deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContracts'))\
    .params(DEPLOY_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0x60403becec73e40eb5a2848791bb4ee5ef0313c70ca05ae766da8406609b9405'

In [57]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0x60403becec73e40eb5a2848791bb4ee5ef0313c70ca05ae766da8406609b9405',
 'blockHeight': 3631032,
 'blockHash': '0x9975815522ecc77b73966747a505256d17710590d72c5f986fe231e878da3970',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cx38d0c023d42c7905ac7b04a8970a0f3ef28800e8',
 'stepUsed': 1195732600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1195732600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [19]:
UPDATE_PARAMS =  {
            "_tokenName": "TestToken1",
            "_symbolName": "TK1",
			"_initialSupply": 1000,
            "_decimals": 18
		}
        
deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContracts'))\
    .params(UPDATE_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0x42d14b373d7ff2dc19b636de4d51dd8008ee79a8405c7f8c41d7618e0d552b8a'

In [21]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0x42d14b373d7ff2dc19b636de4d51dd8008ee79a8405c7f8c41d7618e0d552b8a',
 'blockHeight': 3631512,
 'blockHash': '0x7d14ec899171e474ffa53cc7f48214c99ee6c4f5fe58fa77914601c88979af7e',
 'txIndex': 1,
 'to': 'cx38d0c023d42c7905ac7b04a8970a0f3ef28800e8',
 'scoreAddress': 'cx38d0c023d42c7905ac7b04a8970a0f3ef28800e8',
 'stepUsed': 1792077960,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1792077960,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [13]:
external_methods = ["name", "symbol", "decimals", "totalSupply"]
for method in external_methods:
    call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method)\
                    .build()
    print(icon_service.call(call))

TestToken1
TK1
0x12
0x3635c9adc5dea00000


In [23]:
transfer_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method('transfer')\
    .params({'_to':prajwol.get_address(), '_value':100})\
    .build()

estimate_step = icon_service.estimate_step(transfer_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(transfer_transaction, operator, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0x96e4373935d1333f01d83cb18d7f8688d7e37fb54af5eaabddf8ce1ef11b7f88'

In [24]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0x96e4373935d1333f01d83cb18d7f8688d7e37fb54af5eaabddf8ce1ef11b7f88',
 'blockHeight': 3631521,
 'blockHash': '0xdb559215b199e20d1c406a4a36eb4cea7dfa83bbba8053617a2d4b63e40cc64d',
 'txIndex': 1,
 'to': 'cx38d0c023d42c7905ac7b04a8970a0f3ef28800e8',
 'stepUsed': 153880,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 153880,
 'eventLogs': [{'scoreAddress': 'cx38d0c023d42c7905ac7b04a8970a0f3ef28800e8',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx8b94a3792f336b71937709dae5487166c180c87a',
    'hx49763207da6a6213aaf9a0ea02f8d877fbd0105a',
    '0x64'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00 \x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\

In [25]:
call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'account': prajwol.get_address()})\
                    .build()
print(icon_service.call(call))

0xc8
